In [28]:
#Import Dependencies and Formatting
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
%matplotlib inline

# Reflect Tables into SQLAlchemy ORM

In [29]:
#Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [30]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [31]:
#Declare a Base using automap_base()
Base = automap_base()

#Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [32]:
#Print all of the classes mapped to the Base
Base.classes.keys()

['measurement', 'station']

In [33]:
#Assign the measurement class to a variable called 'measurement'
measurement = Base.classes.measurement

#Assign the station class to a variable called 'station'
station = Base.classes.station

In [34]:
#Create the session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [35]:
#Use the inspector to see the columns of the measurement object
inspector = inspect(engine)

columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [36]:
#Use the inspector to see the columns of the station object
inspector = inspect(engine)

columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [37]:
#Design a query to retrieve the last 12 months of precipitation data and plot the results

#Calculate the date 1 year ago from the last data point in the database
date_query = session.query(measurement.date).order_by(measurement.date.desc()).first()
last_date_str = date_query[0]
last_date = dt.datetime.strptime(last_date_str, '%Y-%m-%d')
    #https://stackoverflow.com/questions/466345/converting-string-into-datetime
year_ago = last_date - dt.timedelta(days=365)
print(f'This shows the percipitation data between {year_ago} and {last_date}.')

#Perform a query to retrieve the data and precipitation scores
precip_query = session.query(measurement.date, measurement.prcp).filter(measurement.date >= year_ago)

#Save the query results as a Pandas DataFrame and set the index to the date column
precip_df = pd.DataFrame(precip_query, columns = ['Date', 'Precipitation'])

#Sort the dataframe by date
precip_df.sort_values('Date')

This shows the percipitation data between 2016-08-23 00:00:00 and 2017-08-23 00:00:00.


,Date,Precipitation
0,2016-08-24,0.08
1006,2016-08-24,NaN
1524,2016-08-24,2.15
702,2016-08-24,2.28
360,2016-08-24,2.15
...,...,...
1522,2017-08-22,0.00
1523,2017-08-23,0.08
359,2017-08-23,0.00
1005,2017-08-23,0.00


In [40]:
#Use Pandas Plotting with Matplotlib to plot the precipitation data

In [39]:
#Use Pandas and Numpy to calcualte the summary statistics for the precipitation data
precip_mean = precip_df['Precipitation'].mean()
precip_median = precip_df['Precipitation'].median()
precip_variance = precip_df['Precipitation'].var()
precip_std = precip_df['Precipitation'].std()
precip_sem = precip_df['Precipitation'].sem()

#Combine the statistical values into a summary table
summary_statistics = pd.DataFrame([precip_mean, precip_median, precip_variance, precip_std, precip_sem],['Mean', 'Median', 'Variance', 'Standard Deviation', 'Standard Error of the Mean'], columns = ['Value'])

#Print summary statistics table
summary_statistics

,Value
Mean,0.176462
Median,0.020000
Variance,0.211865
Standard Deviation,0.460288
Standard Error of the Mean,0.010254


In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
